[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nkeriven/ensta-mt12/blob/main/notebooks/01b_knn/N1_blobs_knn.ipynb)

In [ ]:
# Import modules
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # some 

In [ ]:
# Select random seed
random_state = 1

We use scikit-learn to generate a toy 2D data set (two features $x_1$ and $x_2$)  for binary classification  (two classes) 
 - each sample $(x_1,x_2)$ in the dataset is plotted as a 2D point where the two features $x_1$ and $x_2$ are displayed along the abscissa and ordinate axes respectively
 - the corresponding class label $y$ is displayed as a color mark (e.g., yellow or purple)

In [ ]:
from sklearn.datasets import make_classification
#X are the features (aka inputs, ...), y the labels (aka responses, targets, output...)
X,y = make_classification(n_features=2, n_redundant=0, n_informative=2, n_samples=1000,
                          random_state=random_state, n_clusters_per_class=2,
                         class_sep=.7)

from sklearn.model_selection import train_test_split
# We split the initial set in two sets: one, namely the training set, use for training the model, 
# and one, namely the test set, use to compute the validation error
# -> test_size x n_samples for the test set and n_samples x (1- test_size) for the training set
# where test_size is given as a parameter
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state= random_state )
y_train[y_train==0]=-1
y_test[y_test==0]=-1

# Display the training set
plt.scatter(X_train[:,0], X_train[:,1],c=y_train)
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
#plt.savefig("2d_binary_classif.pdf")

Then a _k nearest neighbors_ algorithm is used to learn the classification function/rule.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) # Creation of the classifier
knn.fit(X_train, y_train) # Model fitting

In [ ]:
# Plot the decision functions
XX, YY = np.meshgrid(np.linspace(X[:,0].min(), X[:,0].max(),200),
                     np.linspace(X[:,1].min(), X[:,1].max(),200))
XY = np.vstack([XX.flatten(), YY.flatten()]).T
yp = knn.predict(XY)
plt.contour(XX,YY,yp.reshape(XX.shape),[0])
plt.scatter(XY[:,0],XY[:,1], c=yp/2+1/4, s=.1)
plt.scatter(X_train[:,0], X_train[:,1], c=y_train)
plt.grid("on")
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')

In [ ]:
XY.shape

## Model complexity

For the KNN, the number of neighbors considered for the decision is the main hyperparemeter to set-up for small to medium size problem. Default value for scikit-learn is set to 5 (do knn? for help). In the following we are going to plot the overall accuracy function of the hyperparameter.

In [ ]:
help(knn)

In [ ]:
neighbors = np.arange(1,40)
errors_train, errors_validation = [], []
for n_ in neighbors:
    knn = KNeighborsClassifier(n_neighbors=n_)
    knn.fit(X_train, y_train)
    
    y_pred = knn.predict(X_train)
    errors_train.append( np.mean( y_train != y_pred) )
    
    y_pred = knn.predict(X_test)
    errors_validation.append( np.mean( y_test != y_pred) )
    
plt.plot(neighbors,errors_train)
plt.plot(neighbors,errors_validation)
plt.legend(["Train", "Test"])
plt.xlabel("Number of neighbors")
plt.ylabel("Misclassification rate")
plt.grid()

### Exercise

 - In view of the results obtained, which values of the number of neighbors (or range of values) seem optimal? Verify that these values give boudaries that seem coherent to you.
 - In this plot, which edges of the curve correspond to the over and under-fitting cases respectively?
 - Why the train error can not be used alone to select the method (here the value of $k$)?
 
It is possible to make the classification task more difficult by reducing the  `class_sep` parameter (default is 1) in the [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) function used to generate the data set. In fact this shrinks the spread between the two classes, which increases the overlapping ratio.
 - What happens when you decrease gradually the `class_sep` parameter from 1 to 0.5  ? In particular, how can you explain the shape of the test error curve? 
 